In [1]:
import time
import pandas as pd
from kid_readout.analysis import archive
from equipment.vdi import zbd
from equipment.srs import lockin
plt.rcParams['font.size'] = 11

/home/local/lib/python2.7/site-packages/matplotlib/__init__.py:1155: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
:0: FutureWarning: IPython widgets are experimental and may change in the future.


updated file list


In [2]:
def zbd_peak_voltage(lockin_rms_voltage):
    return 2**(-1/2) * np.pi * lockin_rms_voltage

def lockin_rms_voltage(zbd_peak_voltage):
    return 2**(1/2) / np.pi * zbd_peak_voltage

def zbd_power(lockin_rms_voltage, optical_frequency=None):
    if optical_frequency is None:
        return zbd_peak_voltage(lockin_rms_voltage) / 2200
    else:
        return zbd_peak_voltage(lockin_rms_voltage) / zbd.ZBD().responsivity(optical_frequency)

In [3]:
l = lockin.SR830('/dev/ttyUSB2', timeout=2)

In [5]:
l.reference_frequency

122.07

In [6]:
z = zbd.ZBD()

In [7]:
l.time_constant_integer_to_seconds[l.time_constant]

0.3

In [8]:
def point(df, l, mickey, minnie):
    l.auto_gain()
    time_constant = l.time_constant_integer_to_seconds[l.time_constant]
    time.sleep(5 * time_constant)
    return df.append({'time': time.time(),
                      'mickey': mickey,
                      'minnie': minnie,
                      'R': l.R,
                     'time_constant': time_constant},
             ignore_index=True)
    return df

In [10]:
broadband = archive.load_archive('/data/readout/archive/2015-05-12_mmw_noise_broadband.npy')
#coherent140 = archive.load_archive('/data/readout/archive/2015-05-18_mmw_140_GHz.npy')
#coherent148 = archive.load_archive('/data/readout/archive/2015-05-19_mmw_148_GHz.npy')


In [11]:
broadband.lockin_rms_voltage.max()

0.043075799999999997

In [12]:
zbd_power(broadband.lockin_rms_voltage.max())

4.3495622015345937e-05

In [11]:
coherent140.lockin_rms_voltage.max()

0.036623200000000002

In [12]:
coherent148.lockin_rms_voltage.max()

0.053199999999999997

In [13]:
coherent156.lockin_rms_voltage.max()

0.070400699999999997

Start with the broadband calibration: 

In [14]:
maximum_linear_power = 10e-6
maximum_lockin_voltage = lockin_rms_voltage(maximum_linear_power * z.responsivity(150e9))
maximum_lockin_voltage

0.011429848404680566

In [15]:
minimum_linear_power = 1e-6
minimum_lockin_voltage = lockin_rms_voltage(minimum_linear_power * z.responsivity(150e9))
minimum_lockin_voltage

0.0011429848404680564

# Preliminary data

In [16]:
H3H3 = 40.16
L6H3 = 8.25
L6L6 = 0.681
H3L6 = 3.63

(L6H3 * H3L6) / (L6L6 * H3H3)

1.0950142162627026

In [17]:
H4H4 = 19.5
L6H4 = 4.677
L6L6 = 0.608
H4L6 = 2.54

(L6H4 * H4L6) / (L6L6 * H4H4)

1.001988866396761

# Data

In [35]:
df = pd.DataFrame(columns=('time', 'mickey', 'minnie', 'R', 'time_constant'))

In [106]:
plt.rcParams['figure.figsize'] = (1.5, 1.5)

In [ ]:
df = point(df, l, 9, 9)

In [11]:
norm = matplotlib.colors.LogNorm
mappable = plt.cm.ScalarMappable(norm(min(df.R), max(df.R)), cmap=plt.cm.coolwarm)
mappable.set_array(np.array(df.R))
for n, row in df.iterrows():
    plt.plot(row.mickey, row.minnie, '.', color=mappable.to_rgba(row.R), markersize=7)
plt.xlim(10, -1)
plt.ylim(10, -1)
plt.xlabel('Mickey')
plt.ylabel('Minnie');

NameError: name 'df' is not defined

In [177]:
df['time_constant'] = 0.3

In [180]:
df.loc[(df.mickey >= 8) & (df.minnie >=8), 'time_constant'] = 1

In [181]:
df

,time,mickey,minnie,R,time_constant
0,1.439860e+09,4,4,0.018714,0.3
1,1.439860e+09,4,3,0.029997,0.3
2,1.439860e+09,3,3,0.038414,0.3
3,1.439860e+09,3,4,0.024460,0.3
4,1.439860e+09,5,4,0.011179,0.3
5,1.439860e+09,6,4,0.004543,0.3
6,1.439860e+09,7,4,0.001236,0.3
7,1.439860e+09,8,4,0.000341,0.3
8,1.439860e+09,9,4,0.000067,0.3
9,1.439860e+09,4,5,0.008307,0.3


In [183]:
#np.save('/home/flanigan/equipment/vdi/calibration.npy', df.to_records())

In [184]:
test = pd.DataFrame().from_records(np.load('/home/flanigan/equipment/vdi/calibration.npy'))

In [185]:
test

,index,time,mickey,minnie,R,time_constant
0,0,1.439860e+09,4,4,0.018714,0.3
1,1,1.439860e+09,4,3,0.029997,0.3
2,2,1.439860e+09,3,3,0.038414,0.3
3,3,1.439860e+09,3,4,0.024460,0.3
4,4,1.439860e+09,5,4,0.011179,0.3
5,5,1.439860e+09,6,4,0.004543,0.3
6,6,1.439860e+09,7,4,0.001236,0.3
7,7,1.439860e+09,8,4,0.000341,0.3
8,8,1.439860e+09,9,4,0.000067,0.3
9,9,1.439860e+09,4,5,0.008307,0.3
